# Project - Apache Spark & Elastichsearch

##### Students:
* Lilia IZRI      (DS)
* Yacine MOKHTARI (DS)
* Alexandre COMBEAU (DS)

##### Report
[PENSER A METTRE UN LIEN ICI]


In [1]:
# pip install tweepy

In [1]:
# Imports
import tweepy
from tweepy.auth import OAuthHandler
from tweepy import Stream
import socket
import json
from geo import get_location_by_address

## I. Processing Twitter as a Stream

### 1. Settings : Insert Twitter developer credentials

In [2]:
# KEYS (Remarque mdr c'est des clés que j'ai trouvées sur un site 🤷🏻‍♀️)
consumer_key    = '3XwgFgsXucaFXOrkZtlwvxV5O'
consumer_secret = 'LNnTcxibKalylVImljDsKfkRqb6WHD8I7hdSYt7Pm7VaCetqZm'
access_token    = '608048932-AM21gcwLORlm5j0514DwPL850byLb7Gs04bPZ6Mn'
access_secret   = 'QtnJ8mV75peV7L7Kd4DZKogMRIxzjk5XFOPtZiz62NoM3'

### 2. Creating a new listener class

In [20]:
# Class that represents our Listener
class TweetsListener(Stream):

    def __init__(self, *args, csocket):
        super().__init__(*args)
        self.client_socket = csocket
    def on_data(self, data):
        try:
            # Extracting interesting fields
            tweet = json.loads( data )
            text = tweet['text']
            user = tweet['user']['screen_name']
            # location = tweet['user']['location'] if tweet['user']['location'] is not None else 'Unknown'
            date = tweet['created_at']
            
            # Handle hashtags (we join the list by adding '#' and separate them by a space ' '
            list_hashtags = tweet['entities']['hashtags']
            hashtags = " ".join(['#' + i['text' ] for i in list_hashtags]) if list_hashtags is not [] else ""
            
            # Handle the location 
            location = get_location_by_address(tweet['user']['location']) 
            latitude, longitude = location if location is not None else get_location_by_address('Unknown')
            print(latitude, longitude)
            
            #  We combine data and metadata to send them 
            #  We add a key ###:field:### so we can split the fields easily 
            #  We remove '\n' from a tweet and put one '\n' between tweets 
            tweet_info = (" ###:field:### user: " + user      + " ###:field:### tweet: "    + text +\
                          " ###:field:### date: " + date      + " ###:field:### lat: "      + latitude + \
                          " ###:field:### lon: "  + longitude + " ###:field:### hashtags: " + hashtags + \
                          " ###:field:### id: "+ tweet['user']['id_str']
                          ).replace('\n', ' ') + '\n'
            
            #print(tweet['user']['location'])
            if(tweet['coordinates'] is not None):
                latitude = tweet["coordinates"]
                print(latitude)
            print(tweet_info)
            
            # Send to socket : We convert this tweet into a bite code (since spark takes easily this kind of data)
            self.client_socket.send(tweet_info.encode('utf-8'))
            return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True

    # Handle errors
    def if_error(self, status):
        print(status)
        return True

The method that captures these tweets in spark (```socketTextStream(port..)```) will concider a text containing a single "\n" as two different entries. <br>
Since we don't want the same tweet to be separated in two differents RDDs, we replace each '\n' by a simple space ' '

###  3. Authenticate our connection using the pre-defined credentials 
In order to begin streaming, we have to implement a function that allows us to authenticate our connection

In [5]:
def send_tweets(c_socket):
    """
        Send tweets to client socket (in our case ✨SPARK✨)
    """
    twtr_stream = TweetsListener(
        consumer_key, consumer_secret,
        access_token, access_secret,
        csocket=c_socket
    )
    twtr_stream.filter(track=['Macron'])#, 'depp', 'macron', 'covid', 'ukraine', 'heard', 'yoda', 'thread', 'sport'])

###  4. Testing connection with client (Spark)
The second step would be to pass the collected tweets to Spark. A way to do that, is to send them in a TCP/socket (that would be the entry point of Spark Streaming)

In [6]:
new_skt = socket.socket()         # initiate a socket object
host = "127.0.0.1"                # address host
port = 5553                       # specifie port 
new_skt.bind((host, port))        # Binding host and port

In [ ]:
print(f"Now listening on port: {port}")
new_skt.listen(5)                 # waiting for client connection
c, addr = new_skt.accept()        # Establish connection with client

print(f"Received request from: {addr}")
send_tweets(c)                    # send tweets to client socket

Now listening on port: 5553
Received request from: ('127.0.0.1', 55762)
33.7489924 -84.3902644
 ###:field:### user: Radlein ###:field:### tweet: RT @Taniel: Last leg of deal was struck today between Melenchon’s left populist party and long-dominant center-left Socialist Party. This i… ###:field:### date: Thu May 05 02:59:34 +0000 2022 ###:field:### lat: 33.7489924 ###:field:### lon: -84.3902644 ###:field:### hashtags:  ###:field:### id: 15982094

44.933143 7.540121
 ###:field:### user: 2022Nemesis ###:field:### tweet: @UPR_Asselineau https://t.co/QWqeuTE6YG ###:field:### date: Thu May 05 02:59:35 +0000 2022 ###:field:### lat: 44.933143 ###:field:### lon: 7.540121 ###:field:### hashtags:  ###:field:### id: 1480197559369056262

34.2331373 -102.4107493
 ###:field:### user: ConflictNews6 ###:field:### tweet: @dna https://t.co/kIHBj9beOQ ###:field:### date: Thu May 05 02:59:36 +0000 2022 ###:field:### lat: 34.2331373 ###:field:### lon: -102.4107493 ###:field:### hashtags:  ###:field:### id: